# **PREDICCIONES PARA UN SOLO PARTICIPANTE**

In [ ]:
print("v.0.0")

In [74]:
from statistics import mean, median
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [75]:
INDEXS = [{"train":[{"start":{"day":2,"hour":0,"minute":0},
                     "end":{"day":20,"hour":22,"minute":0}}
                    ],
           "validation":[{"start":{"day":22,"hour":0,"minute":0},
                     "end":{"day":24,"hour":22,"minute":0}}
                    ],
           "test":[{"start":{"day":26,"hour":0,"minute":0},
                     "end":{"day":27,"hour":22,"minute":0}}
                    ]}
          ]

In [76]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 1
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = True
LOW_DATA = True
SPLIT_INTO_TWO_DAYS = True
MULTI_STEP_FORECAST = False
SPLIT = 0
np.random.seed(42)

In [77]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Individual/"

if LOW_DATA:
    PATH += "LowData/"

In [78]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)


KeyboardInterrupt: 

In [ ]:
NUMBER_OF_PARTICIPANTS = dataX.shape[0]
if MULTI_STEP_FORECAST:
    PREDICTED_HORIZON = 1
    if COMPUTED_OPTION == 0:
        dataX_nuevo = dataX
        JUMP = 1
    elif COMPUTED_OPTION == 1:
        JUMP = 60
        dataX_nuevo = np.ones(shape=(NUMBER_OF_PARTICIPANTS,dataX.shape[1],24))
        for k in range(0,NUMBER_OF_PARTICIPANTS):
            for i in range(0,dataX.shape[1]):
                for j in range(0,24):
                    dataX_nuevo[k,i,j] = np.sum(dataX[k,i,60*j:60*(j+1)])
    else:
        JUMP = 1440
        dataX_nuevo = np.ones(shape=(dataX.shape[0],1))
        for k in range(0,NUMBER_OF_PARTICIPANTS):
            for i in range(0,dataX.shape[0]):
                dataX_nuevo[k,i,:] = np.sum(dataX[k,i,:])

In [ ]:
def change_shape_by_participant(data):
    original_shape = data.shape
    new_shape = (original_shape[0] * original_shape[1], original_shape[2])
    reshaped_array = data.reshape(new_shape)
    return reshaped_array

In [ ]:
if MULTI_STEP_FORECAST:
    dataX_nuevo = dataX_nuevo.transpose(1,0,2)

#We split a test set for testing
# train_test_split(dataX, dataY, test_size=TEST_SIZE)

def calculate_index(time):
  minute_index = time["day"] * 1440 + time["hour"]*60 + time["minute"]
  return minute_index

def get_split(dataX,dataY,index):
  start = calculate_index(index[0]["start"])
  end = calculate_index(index[0]["end"])
  X_split = dataX[:,start:end,:]
  y_split = dataY[:,start:end,:]
  if len(index) > 1:
    for i in range(1,index):
      start = calculate_index(index[i]["start"])
      end = calculate_index(index[i]["end"])
      X_split = np.concatenate(X_split,dataX[:,start:end,:])
      y_split = np.concatenate(y_split,dataY[:,start:end,:])
  return X_split,y_split

def train_test_validation_split(dataX,dataY,indexs):
  X_train,y_train = get_split(dataX,dataY,indexs["train"])
  X_validation,y_validation = get_split(dataX,dataY,indexs["validation"])
  X_test,y_test = get_split(dataX,dataY,indexs["test"])
  return X_train,y_train,X_validation,y_validation,X_test,y_test


X_train,y_train,X_validation,y_validation,X_test,y_test = train_test_validation_split(dataX,dataY,INDEXS[SPLIT])
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

In [ ]:
X_train,y_train,X_validation,y_validation,X_test,y_test = [ i.transpose(1,0,2) for i in
    [X_train,y_train,X_validation,y_validation,X_test,y_test ]
    ]
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

In [ ]:

X_train,y_train,X_validation,y_validation,X_test,y_test = [ change_shape_by_participant(i) for i in
    [X_train,y_train,X_validation,y_validation,X_test,y_test ]
    ]
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

In [ ]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[1]
WINDOW_SIZE = X_test.shape[1]

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(inputs) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128,return_sequences=True, activation="relu")(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

In [ ]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [ ]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train,
            y_train,
            epochs=3,
            verbose=1,
            batch_size=128,
            shuffle=True,
            validation_data=(X_validation, y_validation)
               )

In [ ]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

In [ ]:
print(predictions.shape)
print(y_test.shape)
DATA_BY_PARTICIPANT = int(y_test.shape[0]/25)
poblational_prediction = np.ones(shape=(DATA_BY_PARTICIPANT,HORIZON))
poblational_y_test = np.ones(shape=(DATA_BY_PARTICIPANT,HORIZON))
poblational_X_test = np.ones(shape=(DATA_BY_PARTICIPANT,WINDOW_SIZE))
for i in range(0,DATA_BY_PARTICIPANT):
    poblational_prediction[i,:] = np.sum(np.array(predictions[i::DATA_BY_PARTICIPANT]),axis=0)
    poblational_y_test[i,:] = np.sum(np.array(y_test[i::DATA_BY_PARTICIPANT]),axis=0)
    poblational_X_test[i,:] = np.sum(np.array(X_test[i::DATA_BY_PARTICIPANT]),axis=0)
print(poblational_prediction.shape)
print(poblational_y_test.shape)

print('\033[1m' + "MSE: " + str(mean_squared_error(poblational_y_test,poblational_prediction)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(poblational_y_test,poblational_prediction)) + '\033[0m')
print(np.mean(poblational_y_test))

In [ ]:
SPLIT_INTO_TWO_DAYS = True
if not MULTI_STEP_FORECAST:
  print("Resultados poblacionales cada dos horas")
  print('\033[1m' + "MSE: " + str(mean_squared_error(np.sum(poblational_y_test,axis=1),np.sum(poblational_prediction,axis=1))) + '\033[0m')
  print('\033[1m' + "MAE: " + str(mean_absolute_error(np.sum(poblational_y_test,axis=1),np.sum(poblational_prediction,axis=1))) + '\033[0m')
  print(np.mean(np.sum(poblational_y_test,axis=1)))

  list_of_MAE = [  mean_absolute_error(poblational_prediction[i],poblational_y_test[i]) for i in range(0,len(poblational_y_test)) ]
  list_of_values = sorted(list_of_MAE)
  mean_value = mean(list_of_MAE)
  closest_value = min(list_of_MAE, key=lambda x: abs(x - mean_value))
  # Crear un array de índices
  indices = [list_of_MAE.index(list_of_values[-1]),
             list_of_MAE.index(closest_value) ,
            list_of_MAE.index(list_of_values[0])]
  if COMPUTED_OPTION == 0:
    for i in indices:
      print(list_of_MAE[i])
      plot_predictions_vs_real(poblational_prediction[i],poblational_y_test[i])
  else:
    for i in indices:
        print(list_of_MAE[i])
        END = 24
        STARTED_MINUTE = 0
        previous = np.ones(shape=(24))
        for j in range(0,24):
            previous[j] = np.sum(poblational_X_test[i,:][60*j:60*(j+1)])
        predictions_to_plot = np.ones(shape=(END + HORIZON))
        predictions_to_plot[0:END] = previous[:]
        predictions_to_plot[END:] = poblational_prediction[i,:]
        y_test_to_plot = np.ones(shape=(END + HORIZON))
        y_test_to_plot[0:END] = previous[:]
        y_test_to_plot[END:] = poblational_y_test[i,:]
        plot_predictions_vs_real(predictions_to_plot,y_test_to_plot)

if SPLIT_INTO_TWO_DAYS:
    index = 0
    period = poblational_X_test[::120,:]
    period_results = make_preds(model_LSTM,period)
    period_results_aux = np.array(period_results)
    period_results_to_plot = np.array(period_results_aux).reshape(HORIZON*23)
    y_test_to_plot = poblational_y_test[::120,:].reshape(HORIZON*23)
    plot_predictions_vs_real(predictions=period_results_to_plot,reals=y_test_to_plot)
    print('\033[1m' + "Predicted: " + str(np.sum(period_results_to_plot)) + '\033[0m')
    print('\033[1m' + "Reals: " + str(np.sum(y_test_to_plot)) + '\033[0m')




In [ ]:
if SAVE_RESULTS:
  if READ_LOCAL_DATA:
    file_path = 'Resources/Resultados/Individual/'
  else:
    file_path = "/content/drive/MyDrive/TFG/Resources/Resultados/Individual/"
    if LOW_DATA:
        file_path += "LowData/"

    file_path += "Split"+str(SPLIT)+"/"
    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))